Assignment 2 - Extract the reviews from Dune 2021 using IMDB API
===

*Due: December 6 2022*

Assess which sentiment analysis method (AFINN, VADER, TEXTBLOB) and input (Raw, Preprocessed without stop words, Preprocessed with stop words) correlates better with the user rating from IMDB.

In [1]:
#
#
#